In [1]:
import numpy as np
import random
import os
import json
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import StratifiedKFold

In [2]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(1234)

seed_everything(seed=42)

In [3]:
n_folds=5
fold=0

In [4]:
train = pd.read_json("../input/tig-stainless-steel-304/ss304/train/train.json", orient='index')
train['path'] = "../input/tig-stainless-steel-304/ss304/train/"+train.index
train.reset_index(drop=True, inplace=True)
train.rename(columns = {0:'Label'}, inplace = True)
train

In [5]:
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
oof = []
targets = []
target = "Label"

for fold, (trn_idx, val_idx) in enumerate(
    skf.split(train, train[target])
):
    train.loc[val_idx, "fold"] = int(fold)


train.to_csv("../train.csv", index=False)



In [6]:
train_df = train[train.fold != fold].reset_index(drop=False)
val_df = train[train.fold == fold].reset_index(drop=False)

In [7]:
train_df.describe()

In [8]:
preprocess_input = tf.keras.applications.resnet50.preprocess_input

In [9]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

In [10]:
resnet_layers = tf.keras.applications.resnet50.ResNet50(
    include_top=False,
#     weights='imagenet',
#     input_tensor=None,
    input_shape=(256,256,3),
    pooling="max",
    classes=6
)

In [11]:
inputs = tf.keras.Input(shape=(256, 256, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = resnet_layers(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(6, activation = 'softmax')(x)

model = tf.keras.Model(inputs, outputs)
model.summary()

In [12]:
model.compile(optimizer='adam', loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [13]:
from keras_preprocessing.image import ImageDataGenerator

In [14]:
train_df['Label'] = train_df['Label'].astype("str")

In [20]:
train_datagen=ImageDataGenerator()
train_generator=train_datagen.flow_from_dataframe(dataframe=train_df, directory="./", x_col="path", y_col="Label", class_mode="sparse", target_size=(256,256),color_mode="rgb", batch_size=32)

In [16]:
val_df['Label'] = val_df['Label'].astype("str")

In [21]:
valid_datagen=ImageDataGenerator()
valid_datagen=valid_datagen.flow_from_dataframe(dataframe=val_df, directory="./", x_col="path", y_col="Label", class_mode="sparse", target_size=(256,256),color_mode="rgb", batch_size=32)

In [22]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_datagen.n//valid_datagen.batch_size

In [ ]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_datagen,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=5)

In [ ]:
# model.evaluate_generator(generator=valid_generator,
# steps=STEP_SIZE_TEST)test_generator.reset()
# pred=model.predict_generator(test_generator,
# steps=STEP_SIZE_TEST,
# verbose=1)
# predicted_class_indices=np.argmax(pred,axis=1)
# labels = (train_generator.class_indices)
# labels = dict((v,k) for k,v in labels.items())
# predictions = [labels[k] for k in predicted_class_indices]
# filenames=test_generator.filenames
# results=pd.DataFrame({"Filename":filenames,
#                       "Predictions":predictions})
# results.to_csv("results.csv",index=False)

In [ ]:
# fig, axs = plt.subplots(2, 1, figsize=(15,15))
# axs[0].plot(history.history['loss'])
# axs[0].plot(history.history['val_loss'])
# axs[0].title.set_text('Training Loss vs Validation Loss')
# axs[0].set_xlabel('Epochs')
# axs[0].set_ylabel('Loss')
# axs[0].legend(['Train','Val'])
# axs[1].plot(history.history['accuracy'])
# axs[1].plot(history.history['val_accuracy'])
# axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
# axs[1].set_xlabel('Epochs')
# axs[1].set_ylabel('Accuracy')
# axs[1].legend(['Train', 'Val'])